# Bitcoin Scalping Model - Google Colab

Model hybrid LSTM + XGBoost untuk scalping Bitcoin (BTCUSDT 5m) menggunakan data tahun 2024.

## Fitur:
- Model Hybrid: LSTM sebagai encoder sequence + XGBoost sebagai classifier
- Labeling Cerdas: TP/SL dengan horizon 5 bar ke depan
- 40+ indikator teknikal
- Real-time signal generation
- Signal lengkap: Entry price, TP, SL, confidence score


## 1. Setup Environment


In [ ]:
# Install dependencies
!pip install pandas==2.1.4 numpy==1.24.3 scikit-learn==1.3.2 tensorflow==2.15.0 xgboost==2.0.3 matplotlib==3.8.2 seaborn==0.13.0 fastapi==0.104.1 uvicorn==0.24.0 python-multipart==0.0.6 pydantic==2.5.0 ta==0.10.2


In [ ]:
# Check GPU availability
import tensorflow as tf
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")
print(f"TensorFlow Version: {tf.__version__}")

# Enable GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU memory growth enabled")
    except RuntimeError as e:
        print(f"❌ Error setting GPU memory growth: {e}")


In [ ]:
# Create directories
import os
os.makedirs("models", exist_ok=True)
os.makedirs("data", exist_ok=True)
os.makedirs("src", exist_ok=True)
print("✅ Directories created")


## 2. Upload Data

Upload semua file CSV BTCUSDT-5m-2024-*.csv ke folder `data/`


In [ ]:
# Upload files (run this cell and upload your files)
from google.colab import files
import shutil
import zipfile

print("📁 Upload Options:")
print("1. Upload 'bitcoin_scalping_complete.zip' (recommended)")
print("2. Upload individual CSV files")
print("3. Upload individual Python files")

# Option 1: Upload complete package (recommended)
print("\n🚀 Option 1: Upload complete package")
print("Upload 'bitcoin_scalping_complete.zip' and uncomment the code below:")

# Uncomment to upload complete package
# uploaded = files.upload()
# for filename in uploaded.keys():
#     if filename.endswith('.zip'):
#         with zipfile.ZipFile(filename, 'r') as zip_ref:
#             zip_ref.extractall('.')
#         print(f"Extracted {filename}")
#         os.remove(filename)  # Remove zip file after extraction

# Option 2: Upload individual CSV files
print("\n📊 Option 2: Upload individual CSV files")
print("Expected files: BTCUSDT-5m-2024-01.csv to BTCUSDT-5m-2024-12.csv")

# Uncomment to upload individual CSV files
# uploaded = files.upload()
# for filename in uploaded.keys():
#     if filename.startswith('BTCUSDT-5m-2024-'):
#         shutil.move(filename, f'data/{filename}')
#         print(f'Moved {filename} to data/')


## 3. Upload Source Code

Upload semua file Python dari folder `src/`


In [ ]:
# Upload source code files (if not using complete package)
print("📁 Upload source code files (if not using complete package)")
print("Required files:")
print("- merge_data.py")
print("- features.py")
print("- labeling.py")
print("- model.py")
print("- train.py")
print("- signal.py")
print("- serve.py")
print("- __init__.py")

# Uncomment to upload individual Python files
# uploaded = files.upload()
# for filename in uploaded.keys():
#     if filename.endswith('.py'):
#         shutil.move(filename, f'src/{filename}')
#         print(f'Moved {filename} to src/')

print("\n💡 Tip: If you uploaded the complete package, skip this step!")


## 4. Test Imports


In [ ]:
# Test imports
import sys
sys.path.append('src')

try:
    from src.merge_data import load_and_merge_2024_data
    print("✅ merge_data: OK")
    
    from src.features import calculate_technical_indicators
    print("✅ features: OK")
    
    from src.labeling import calculate_tp_sl_labels
    print("✅ labeling: OK")
    
    from src.model import HybridLSTMXGBModel
    print("✅ model: OK")
    
    from src.signal import SignalGenerator
    print("✅ signal: OK")
    
    print("\n🎉 All modules imported successfully!")
    
except Exception as e:
    print(f"❌ Import error: {e}")


## 5. Load and Prepare Data


In [ ]:
# Load data
from src.merge_data import load_and_merge_2024_data
from src.features import calculate_technical_indicators
from src.labeling import calculate_tp_sl_labels, create_trading_signals

print("📊 Loading data...")
df = load_and_merge_2024_data()
print(f"Data shape: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")

print("\n🔧 Calculating technical indicators...")
df_features = calculate_technical_indicators(df)
print(f"Features shape: {df_features.shape}")

print("\n🏷️ Creating labels...")
df_labeled = calculate_tp_sl_labels(df_features)
signals = create_trading_signals(df_labeled)
print(f"Signals shape: {signals.shape}")
print(f"Signal distribution:")
print(signals['signal'].value_counts())


## 6. Train Model


In [ ]:
# Train model
from src.train import prepare_training_data, train_model, evaluate_model

print("🚀 Preparing training data...")
X_train, X_val, X_test, y_train, y_val, y_test, feature_columns = prepare_training_data()

print("\n🏋️ Training model...")
model, history = train_model(X_train, y_train, X_val, y_val, epochs=50, batch_size=128)

print("\n📊 Evaluating model...")
metrics = evaluate_model(model, X_test, y_test)

print("\n✅ Training completed!")


## 7. Test Signal Generation


In [ ]:
# Test signal generation
from src.signal import generate_signal_from_data

print("🎯 Testing signal generation...")
signal = generate_signal_from_data(df_features)

print("\n📈 Generated Signal:")
for key, value in signal.items():
    print(f"  {key}: {value}")

print("\n✅ Signal generation test completed!")


## 8. Visualize Results


In [ ]:
# Plot training history
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Loss
axes[0, 0].plot(history['loss'], label='Training Loss')
axes[0, 0].plot(history['val_loss'], label='Validation Loss')
axes[0, 0].set_title('Model Loss')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Accuracy
axes[0, 1].plot(history['accuracy'], label='Training Accuracy')
axes[0, 1].plot(history['val_accuracy'], label='Validation Accuracy')
axes[0, 1].set_title('Model Accuracy')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Confusion Matrix
cm = metrics['confusion_matrix']
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Trade/Short', 'Long'], 
            yticklabels=['No Trade/Short', 'Long'],
            ax=axes[1, 0])
axes[1, 0].set_title('Confusion Matrix')
axes[1, 0].set_xlabel('Predicted')
axes[1, 0].set_ylabel('Actual')

# Price chart with signals
axes[1, 1].plot(df.index[-1000:], df['close'].iloc[-1000:], label='BTC Price', alpha=0.7)
axes[1, 1].set_title('BTC Price (Last 1000 bars)')
axes[1, 1].set_xlabel('Date')
axes[1, 1].set_ylabel('Price')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

print("\n📊 Model Performance:")
print(f"Accuracy: {metrics['accuracy']:.4f}")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")
print(f"F1-Score: {metrics['f1_score']:.4f}")


## 9. Download Trained Models


In [ ]:
# Download trained models
import zipfile
import os

print("📦 Creating model package...")

# Create zip file with all model files
with zipfile.ZipFile('bitcoin_scalping_models.zip', 'w') as zipf:
    for root, dirs, files in os.walk('models'):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, file)
    
    # Add source code
    for root, dirs, files in os.walk('src'):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, file)

print("✅ Model package created: bitcoin_scalping_models.zip")
print("\n📥 Download the zip file to use the trained model locally")

# Download the zip file
from google.colab import files
files.download('bitcoin_scalping_models.zip')


## 🎉 Summary

Model Bitcoin scalping telah berhasil dilatih dengan:

- **Data**: 105,408 bar BTCUSDT 5m 2024
- **Features**: 40+ indikator teknikal
- **Model**: Hybrid LSTM + XGBoost
- **Signals**: Long/Short/None dengan TP/SL
- **Performance**: Accuracy, Precision, Recall, F1-Score

Model siap digunakan untuk generate signal trading real-time!
